In [ ]:
import os
import glob
import time
import torch
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset

import torchvision.models as models
from torchvision import transforms, utils

from PIL import Image
from skimage.transform import resize

import warnings
warnings.filterwarnings("ignore")

### Load Dataset

In [ ]:
class dataset(Dataset):
    def __init__(self, root_dir, dataset_type, img_size, transform=None, shuffle=False):
        self.root_dir = root_dir
        self.transform = transform
        self.file_names = [f for f in glob.glob(os.path.join(root_dir, "*", "*.npz")) \
                            if dataset_type in f]
        self.img_size = img_size
        self.shuffle = shuffle

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        data_path = self.file_names[idx]
        data = np.load(data_path)
        image = data["image"].reshape(16, 160, 160)
        target = data["target"]
        structure = data["structure"]
        meta_target = data["meta_target"]
        meta_structure = data["meta_structure"]

        if self.shuffle:
            context = image[:8, :, :]
            choices = image[8:, :, :]
            indices = list(range(8))
            np.random.shuffle(indices)
            new_target = indices.index(target)
            new_choices = choices[indices, :, :]
            image = np.concatenate((context, new_choices))
            target = new_target
        
        resize_image = []
        for idx in range(0, 16):
            resize_image.append(resize(image[idx,:,:], (self.img_size, self.img_size)))
        resize_image = np.stack(resize_image)

        embedding = torch.zeros((6, 300), dtype=torch.float)
        indicator = torch.zeros(1, dtype=torch.float)
        element_idx = 0
    
        del data
        if self.transform:
            resize_image = self.transform(resize_image)
            # meta_matrix = self.transform(meta_matrix)
            target = torch.tensor(target, dtype=torch.long)
            meta_target = self.transform(meta_target)
            meta_structure = self.transform(meta_structure)
            # meta_target = torch.tensor(meta_target, dtype=torch.long)
        return resize_image, target, meta_target, meta_structure, embedding, indicator

#### Experiment Hyperparameters

In [ ]:
class Args:
    
    def __init__(self,):
        self.model = 'CNN_MLP'
        self.epochs = 100
        self.batch_size = 128
        self.seed = 12345
        self.device = 0
        self.load_workers = 80
        self.resume = False
        self.path = '/common/home/ls1274/data_pri/ps851'
        self.save = './ckpt_res/'
        self.img_size = 80
        self.lr = 1e-4
        self.beta1 = 0.9
        self.beta2 = 0.999
        self.epsilon = 1e-8
        self.meta_alpha = 0.
        self.meta_beta = 0.
        self.perc_train = 80
        
args = Args()

args.cuda = torch.cuda.is_available()
torch.cuda.set_device(args.device)
torch.cuda.manual_seed(args.seed)

if not os.path.exists(args.save):
    os.makedirs(args.save)

#### Load Data

In [ ]:
def gen():
    file_list = []
    for root, dir, files in os.walk("/common/home/kp956/CS536/I-RAVEN/test/"):
        for file in files:
            if file.endswith(".npz"):
                file_list.append(os.path.join(root,file))
    return file_list

class ToTensor(object):
    def __call__(self, sample):
        return torch.tensor(sample, dtype=torch.float32)

train = dataset(args.path, "train", args.img_size, transform=transforms.Compose([ToTensor()]),shuffle=True)
valid = dataset(args.path, "val", args.img_size, transform=transforms.Compose([ToTensor()]))
test = dataset(args.path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))

subset_indices = np.random.choice(len(train), len(train)*args.perc_train // 100, replace=False)
train_subset = Subset(train, subset_indices)

print("Number of samples in original train set =", len(train))
print("Number of samples in train subset =", len(train_subset))
print("All samples are unique =", len(subset_indices) == len(set(subset_indices)))

trainloader = DataLoader(train_subset, batch_size=args.batch_size, shuffle=False, num_workers=80)
validloader = DataLoader(valid, batch_size=args.batch_size, shuffle=False, num_workers=80)
testloader = DataLoader(test, batch_size=args.batch_size, shuffle=False, num_workers=80)

Number of samples in original train set = 42000
Number of samples in train subset = 33600
All samples are unique = True


### Model

In [ ]:
class BasicModel(nn.Module):
    def __init__(self, args):
        super(BasicModel, self).__init__()
        self.name = args.model
    
    def load_model(self, path, epoch):
        state_dict = torch.load(path+'{}_epoch_{}.pth'.format(self.name, epoch))['state_dict']
        self.load_state_dict(state_dict)

    def save_model(self, path, epoch, acc, loss):
        torch.save({'state_dict': self.state_dict(), 'acc': acc, 'loss': loss}, path+'{}_epoch_{}.pth'.format(self.name, epoch))

    def compute_loss(self, output, target, meta_target, meta_structure):
        pass

    def train_(self, image, target, meta_target, meta_structure, embedding, indicator):
        self.optimizer.zero_grad()
        output = self(image, embedding, indicator)
        loss = self.compute_loss(output, target, meta_target, meta_structure)
        loss.backward()
        self.optimizer.step()
        pred = output[0].data.max(1)[1]
        correct = pred.eq(target.data).cpu().sum().numpy()
        accuracy = correct * 100.0 / target.size()[0]
        return loss.item(), accuracy

    def validate_(self, image, target, meta_target, meta_structure, embedding, indicator):
        with torch.no_grad():
            output = self(image, embedding, indicator)
        loss = self.compute_loss(output, target, meta_target, meta_structure)
        pred = output[0].data.max(1)[1]
        correct = pred.eq(target.data).cpu().sum().numpy()
        accuracy = correct * 100.0 / target.size()[0]
        return loss.item(), accuracy

    def test_(self, image, target, meta_target, meta_structure, embedding, indicator):
        with torch.no_grad():
            output = self(image, embedding, indicator)
        pred = output[0].data.max(1)[1]
        correct = pred.eq(target.data).cpu().sum().numpy()
        accuracy = correct * 100.0 / target.size()[0]
        return accuracy

#### CNN MLP

In [ ]:
class conv_module(nn.Module):
    def __init__(self):
        super(conv_module, self).__init__()
        self.conv1 = nn.Conv2d(16, 32, kernel_size=3, stride=2)
        self.batch_norm1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=2)
        self.batch_norm2 = nn.BatchNorm2d(32)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=2)
        self.batch_norm3 = nn.BatchNorm2d(32)
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(32, 32, kernel_size=3, stride=2)
        self.batch_norm4 = nn.BatchNorm2d(32)
        self.relu4 = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(self.batch_norm1(x))
        x = self.conv2(x)
        x = self.relu2(self.batch_norm2(x))
        x = self.conv3(x)
        x = self.relu3(self.batch_norm3(x))
        x = self.conv4(x)
        x = self.relu4(self.batch_norm4(x))
        return x.view(-1, 32*4*4)

class mlp_module(nn.Module):
    def __init__(self):
        super(mlp_module, self).__init__()
        self.fc1 = nn.Linear(32*4*4, 512)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(512, 8)
        self.dropout = nn.Dropout(0.5)
        self.sf = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.sf(x)
        return x

class CNN_MLP(BasicModel):
    def __init__(self, args):
        super(CNN_MLP, self).__init__(args)
        self.conv = conv_module()
        self.mlp = mlp_module()
        #self.fc_tree_net = FCTreeNet(in_dim=300, img_dim=512)
        self.optimizer = optim.Adam(self.parameters(), lr=args.lr, betas=(args.beta1, args.beta2), eps=args.epsilon)

    def compute_loss(self, output, target, meta_target, meta_structure):
        pred = output[0]
        loss = F.cross_entropy(pred, target)
        return loss

    def forward(self, x, embedding, indicator):
        alpha = 0
        #x = cv2.resize(x,dsize=(-1,16,80,80))
        features = self.conv(x.view(-1, 16, 80, 80))
        features_tree = features.view(-1, 1, 512)
        #features_tree = self.fc_tree_net(features_tree, embedding, indicator)
        final_features = features + alpha #* features_tree
        score = self.mlp(final_features)
        return score, None

### Training and Evalution

In [ ]:
### Helper functions

def train(epoch, save_file):
    model.train()
    train_loss = 0
    accuracy = 0

    loss_all = 0.0
    acc_all = 0.0
    counter = 0
    for batch_idx, (image, target, meta_target, meta_structure, embedding, indicator) in enumerate(trainloader):
        counter += 1
        if args.cuda:
            image = image.cuda()
            target = target.cuda()
            meta_target = meta_target.cuda()
            meta_structure = meta_structure.cuda()
            embedding = embedding.cuda()
            indicator = indicator.cuda()
        loss, acc = model.train_(image, target, meta_target, meta_structure, embedding, indicator)
        save_str = 'Train: Epoch:{}, Batch:{}, Loss:{:.6f}, Acc:{:.4f}'.format(epoch, batch_idx, loss, acc)
        if counter % 20 == 0:
            print(save_str)
        with open(save_file, 'a') as f:
            f.write(save_str + "\n")
        loss_all += loss
        acc_all += acc
    if counter > 0:
        save_str = "Train_: Avg Training Loss: {:.6f}, Avg Training Acc: {:.6f}".format(
            loss_all/float(counter),
            acc_all/float(counter)
        )
        print(save_str)
        with open(save_file, 'a') as f:
            f.write(save_str + "\n")
    return loss_all/float(counter), acc_all/float(counter)

def validate(epoch, save_file):
    model.eval()
    val_loss = 0
    accuracy = 0

    loss_all = 0.0
    acc_all = 0.0
    counter = 0
    batch_idx = 0
    for batch_idx, (image, target, meta_target, meta_structure, embedding, indicator) in enumerate(validloader):
        #print(image)
        counter += 1
        if args.cuda:
            image = image.cuda()
            target = target.cuda()
            meta_target = meta_target.cuda()
            meta_structure = meta_structure.cuda()
            embedding = embedding.cuda()
            indicator = indicator.cuda()
        loss, acc = model.validate_(image, target, meta_target, meta_structure, embedding, indicator)
        # print('Validate: Epoch:{}, Batch:{}, Loss:{:.6f}, Acc:{:.4f}.'.format(epoch, batch_idx, loss, acc)) 
        loss_all += loss
        acc_all += acc
    if counter > 0:
        save_str = "Val_: Total Validation Loss: {:.6f}, Acc: {:.4f}".format(loss_all/float(counter), acc_all/float(counter))
        print(save_str)
        with open(save_file, 'a') as f:
            f.write(save_str + "\n")
    return loss_all/float(counter), acc_all/float(counter)

def test(epoch, save_file):
    model.eval()
    accuracy = 0

    acc_all = 0.0
    counter = 0
    for batch_idx, (image, target, meta_target, meta_structure, embedding, indicator) in enumerate(testloader):
        counter += 1
        if args.cuda:
            image = image.cuda()
            target = target.cuda()
            meta_target = meta_target.cuda()
            meta_structure = meta_structure.cuda()
            embedding = embedding.cuda()
            indicator = indicator.cuda()
        acc = model.test_(image, target, meta_target, meta_structure, embedding, indicator)
        # print('Test: Epoch:{}, Batch:{}, Acc:{:.4f}.'.format(epoch, batch_idx, acc))  
        acc_all += acc
    if counter > 0:
        save_str = "Test_: Total Testing Acc: {:.4f}".format(acc_all / float(counter))
        print(save_str)
        with open(save_file, 'a') as f:
            f.write(save_str + "\n")
    return acc_all/float(counter)

In [ ]:
model = CNN_MLP(args)
model = model.cuda()

In [ ]:
SAVE_FILE = "last_take1_cnn_mlp_" + time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime()) + "_" + str(args.perc_train)

for epoch in range(0, args.epochs):
    t0 = time.time()
    train(epoch, SAVE_FILE)
    avg_loss, avg_acc = validate(epoch, SAVE_FILE)
    test_acc = test(epoch, SAVE_FILE)
    model.save_model(args.save, epoch, avg_acc, avg_loss)
    print("Time taken = {:.4f} s\n".format(time.time() - t0))

Train: Epoch:0, Batch:19, Loss:2.081057, Acc:7.8125
Train: Epoch:0, Batch:39, Loss:2.079684, Acc:12.5000
Train: Epoch:0, Batch:59, Loss:2.078340, Acc:7.8125
Train: Epoch:0, Batch:79, Loss:2.084720, Acc:9.3750
Train: Epoch:0, Batch:99, Loss:2.079900, Acc:14.8438
Train: Epoch:0, Batch:119, Loss:2.082559, Acc:9.3750
Train: Epoch:0, Batch:139, Loss:2.078937, Acc:14.0625
Train: Epoch:0, Batch:159, Loss:2.077233, Acc:16.4062
Train: Epoch:0, Batch:179, Loss:2.077594, Acc:15.6250
Train_: Avg Training Loss: 2.079825, Avg Training Acc: 12.433511
Val_: Total Validation Loss: 2.078705, Acc: 13.8393
Test_: Total Testing Acc: 12.6488
Time taken = 76.2798 s

Train: Epoch:1, Batch:19, Loss:2.077242, Acc:10.9375
Train: Epoch:1, Batch:39, Loss:2.077981, Acc:10.1562
Train: Epoch:1, Batch:59, Loss:2.077896, Acc:12.5000
Train: Epoch:1, Batch:79, Loss:2.078767, Acc:17.1875
Train: Epoch:1, Batch:99, Loss:2.078537, Acc:14.8438
Train: Epoch:1, Batch:119, Loss:2.076161, Acc:14.8438
Train: Epoch:1, Batch:139, Lo

Train: Epoch:12, Batch:139, Loss:1.843377, Acc:44.5312
Train: Epoch:12, Batch:159, Loss:1.881179, Acc:39.8438
Train: Epoch:12, Batch:179, Loss:1.941699, Acc:32.8125
Train_: Avg Training Loss: 1.896126, Avg Training Acc: 38.842254
Val_: Total Validation Loss: 2.031675, Acc: 21.8874
Test_: Total Testing Acc: 22.2098
Time taken = 79.7777 s

Train: Epoch:13, Batch:19, Loss:1.872465, Acc:43.7500
Train: Epoch:13, Batch:39, Loss:1.884974, Acc:41.4062
Train: Epoch:13, Batch:59, Loss:1.878592, Acc:39.8438
Train: Epoch:13, Batch:79, Loss:1.964079, Acc:28.9062
Train: Epoch:13, Batch:99, Loss:1.880256, Acc:41.4062
Train: Epoch:13, Batch:119, Loss:1.898038, Acc:38.2812
Train: Epoch:13, Batch:139, Loss:1.796030, Acc:52.3438
Train: Epoch:13, Batch:159, Loss:1.864965, Acc:39.8438
Train: Epoch:13, Batch:179, Loss:1.924455, Acc:34.3750
Train_: Avg Training Loss: 1.883212, Avg Training Acc: 40.263464
Val_: Total Validation Loss: 2.019605, Acc: 23.0903
Test_: Total Testing Acc: 23.3755
Time taken = 79.348

Val_: Total Validation Loss: 2.040427, Acc: 21.8874
Test_: Total Testing Acc: 21.6518
Time taken = 82.8372 s

Train: Epoch:25, Batch:19, Loss:1.759315, Acc:53.9062
Train: Epoch:25, Batch:39, Loss:1.737641, Acc:57.0312
Train: Epoch:25, Batch:59, Loss:1.736269, Acc:55.4688
Train: Epoch:25, Batch:79, Loss:1.819627, Acc:46.8750
Train: Epoch:25, Batch:99, Loss:1.778072, Acc:50.7812
Train: Epoch:25, Batch:119, Loss:1.768528, Acc:53.1250
Train: Epoch:25, Batch:139, Loss:1.689016, Acc:59.3750
Train: Epoch:25, Batch:159, Loss:1.723711, Acc:57.0312
Train: Epoch:25, Batch:179, Loss:1.781945, Acc:50.7812
Train_: Avg Training Loss: 1.752324, Avg Training Acc: 53.781582
Val_: Total Validation Loss: 2.047375, Acc: 21.2798
Test_: Total Testing Acc: 21.0193
Time taken = 90.8567 s

Train: Epoch:26, Batch:19, Loss:1.729862, Acc:54.6875
Train: Epoch:26, Batch:39, Loss:1.736122, Acc:56.2500
Train: Epoch:26, Batch:59, Loss:1.717925, Acc:57.0312
Train: Epoch:26, Batch:79, Loss:1.816161, Acc:46.0938
Train: Ep

Train: Epoch:37, Batch:59, Loss:1.666594, Acc:61.7188
Train: Epoch:37, Batch:79, Loss:1.740948, Acc:54.6875
Train: Epoch:37, Batch:99, Loss:1.754720, Acc:52.3438
Train: Epoch:37, Batch:119, Loss:1.696290, Acc:60.1562
Train: Epoch:37, Batch:139, Loss:1.618030, Acc:67.1875
Train: Epoch:37, Batch:159, Loss:1.646267, Acc:64.8438
Train: Epoch:37, Batch:179, Loss:1.662224, Acc:62.5000
Train_: Avg Training Loss: 1.683868, Avg Training Acc: 60.322473
Val_: Total Validation Loss: 2.046154, Acc: 21.5030
Test_: Total Testing Acc: 21.0813
Time taken = 78.9997 s

Train: Epoch:38, Batch:19, Loss:1.673570, Acc:61.7188
Train: Epoch:38, Batch:39, Loss:1.667648, Acc:62.5000
Train: Epoch:38, Batch:59, Loss:1.673913, Acc:61.7188
Train: Epoch:38, Batch:79, Loss:1.743267, Acc:53.9062
Train: Epoch:38, Batch:99, Loss:1.706766, Acc:57.8125
Train: Epoch:38, Batch:119, Loss:1.691883, Acc:59.3750
Train: Epoch:38, Batch:139, Loss:1.620520, Acc:66.4062
Train: Epoch:38, Batch:159, Loss:1.650664, Acc:65.6250
Train: E

Train: Epoch:49, Batch:139, Loss:1.579322, Acc:70.3125
Train: Epoch:49, Batch:159, Loss:1.638620, Acc:64.0625
Train: Epoch:49, Batch:179, Loss:1.644949, Acc:64.0625
Train_: Avg Training Loss: 1.642555, Avg Training Acc: 64.091589
Val_: Total Validation Loss: 2.034305, Acc: 22.5198
Test_: Total Testing Acc: 21.8998
Time taken = 89.3559 s

Train: Epoch:50, Batch:19, Loss:1.653753, Acc:64.0625
Train: Epoch:50, Batch:39, Loss:1.615532, Acc:67.1875
Train: Epoch:50, Batch:59, Loss:1.620070, Acc:67.1875
Train: Epoch:50, Batch:79, Loss:1.698067, Acc:57.0312
Train: Epoch:50, Batch:99, Loss:1.687890, Acc:58.5938
Train: Epoch:50, Batch:119, Loss:1.676803, Acc:60.1562
Train: Epoch:50, Batch:139, Loss:1.575843, Acc:71.0938
Train: Epoch:50, Batch:159, Loss:1.607294, Acc:67.1875
Train: Epoch:50, Batch:179, Loss:1.621190, Acc:66.4062
Train_: Avg Training Loss: 1.641864, Avg Training Acc: 64.133145
Val_: Total Validation Loss: 2.039017, Acc: 21.9618
Test_: Total Testing Acc: 21.8378
Time taken = 86.564

Val_: Total Validation Loss: 2.049494, Acc: 21.0441
Test_: Total Testing Acc: 21.3542
Time taken = 77.2419 s

Train: Epoch:62, Batch:19, Loss:1.613592, Acc:66.4062
Train: Epoch:62, Batch:39, Loss:1.589994, Acc:68.7500
Train: Epoch:62, Batch:59, Loss:1.593979, Acc:68.7500
Train: Epoch:62, Batch:79, Loss:1.649969, Acc:63.2812
Train: Epoch:62, Batch:99, Loss:1.649912, Acc:61.7188
Train: Epoch:62, Batch:119, Loss:1.645314, Acc:63.2812
Train: Epoch:62, Batch:139, Loss:1.542405, Acc:74.2188
Train: Epoch:62, Batch:159, Loss:1.584534, Acc:69.5312
Train: Epoch:62, Batch:179, Loss:1.616014, Acc:66.4062
Train_: Avg Training Loss: 1.616460, Avg Training Acc: 66.335605
Val_: Total Validation Loss: 2.042958, Acc: 21.6518
Test_: Total Testing Acc: 22.1726
Time taken = 79.0004 s

Train: Epoch:63, Batch:19, Loss:1.623334, Acc:65.6250
Train: Epoch:63, Batch:39, Loss:1.596256, Acc:69.5312
Train: Epoch:63, Batch:59, Loss:1.603688, Acc:67.9688
Train: Epoch:63, Batch:79, Loss:1.651578, Acc:62.5000
Train: Ep

Train: Epoch:74, Batch:59, Loss:1.573433, Acc:71.0938
Train: Epoch:74, Batch:79, Loss:1.636371, Acc:63.2812
Train: Epoch:74, Batch:99, Loss:1.632158, Acc:66.4062
Train: Epoch:74, Batch:119, Loss:1.597042, Acc:68.7500
Train: Epoch:74, Batch:139, Loss:1.515709, Acc:77.3438
Train: Epoch:74, Batch:159, Loss:1.557725, Acc:72.6562
Train: Epoch:74, Batch:179, Loss:1.598229, Acc:67.1875
Train_: Avg Training Loss: 1.587234, Avg Training Acc: 69.680851
Val_: Total Validation Loss: 2.039424, Acc: 22.1354
Test_: Total Testing Acc: 21.6022
Time taken = 86.2957 s

Train: Epoch:75, Batch:19, Loss:1.572470, Acc:71.0938
Train: Epoch:75, Batch:39, Loss:1.561753, Acc:71.8750
Train: Epoch:75, Batch:59, Loss:1.562480, Acc:71.0938
Train: Epoch:75, Batch:79, Loss:1.631020, Acc:64.8438
Train: Epoch:75, Batch:99, Loss:1.629848, Acc:65.6250
Train: Epoch:75, Batch:119, Loss:1.586914, Acc:71.0938
Train: Epoch:75, Batch:139, Loss:1.519110, Acc:76.5625
Train: Epoch:75, Batch:159, Loss:1.566677, Acc:71.0938
Train: E

Train: Epoch:86, Batch:139, Loss:1.493686, Acc:78.9062
Train: Epoch:86, Batch:159, Loss:1.569911, Acc:71.0938
Train: Epoch:86, Batch:179, Loss:1.553369, Acc:72.6562
Train_: Avg Training Loss: 1.561694, Avg Training Acc: 72.298870
Val_: Total Validation Loss: 2.044172, Acc: 21.8006
Test_: Total Testing Acc: 21.2922
Time taken = 79.5949 s

Train: Epoch:87, Batch:19, Loss:1.560818, Acc:71.0938
Train: Epoch:87, Batch:39, Loss:1.530679, Acc:75.7812
Train: Epoch:87, Batch:59, Loss:1.532990, Acc:74.2188
Train: Epoch:87, Batch:79, Loss:1.597371, Acc:68.7500
Train: Epoch:87, Batch:99, Loss:1.595440, Acc:67.9688
Train: Epoch:87, Batch:119, Loss:1.562101, Acc:71.8750
Train: Epoch:87, Batch:139, Loss:1.501525, Acc:79.6875
Train: Epoch:87, Batch:159, Loss:1.559100, Acc:71.8750
Train: Epoch:87, Batch:179, Loss:1.564457, Acc:71.0938
Train_: Avg Training Loss: 1.557850, Avg Training Acc: 72.859874
Val_: Total Validation Loss: 2.042136, Acc: 22.0858
Test_: Total Testing Acc: 21.5154
Time taken = 79.520

Val_: Total Validation Loss: 2.051269, Acc: 21.1558
Test_: Total Testing Acc: 20.8829
Time taken = 77.3465 s

Train: Epoch:99, Batch:19, Loss:1.541678, Acc:75.0000
Train: Epoch:99, Batch:39, Loss:1.511177, Acc:77.3438
Train: Epoch:99, Batch:59, Loss:1.525406, Acc:76.5625
Train: Epoch:99, Batch:79, Loss:1.557466, Acc:73.4375
Train: Epoch:99, Batch:99, Loss:1.552861, Acc:75.0000
Train: Epoch:99, Batch:119, Loss:1.554376, Acc:73.4375
Train: Epoch:99, Batch:139, Loss:1.474414, Acc:80.4688
Train: Epoch:99, Batch:159, Loss:1.530183, Acc:74.2188
Train: Epoch:99, Batch:179, Loss:1.534207, Acc:75.0000
Train_: Avg Training Loss: 1.538366, Avg Training Acc: 74.680020
Val_: Total Validation Loss: 2.046631, Acc: 21.1186
Test_: Total Testing Acc: 21.4410
Time taken = 78.3831 s



In [ ]:
# Re-running for 20% of the data
SAVE_FILE = "last_take1_cnn_mlp_20DAT_" + time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime()) + "_" + str(args.perc_train)

for epoch in range(0, args.epochs):
    t0 = time.time()
    train(epoch, SAVE_FILE)
    avg_loss, avg_acc = validate(epoch, SAVE_FILE)
    test_acc = test(epoch, SAVE_FILE)
    model.save_model(args.save, epoch, avg_acc, avg_loss)
    print("Time taken = {:.4f} s\n".format(time.time() - t0))

Train: Epoch:0, Batch:19, Loss:2.079537, Acc:14.0625
Train_: Avg Training Loss: 2.080386, Avg Training Acc: 12.458882
Val_: Total Validation Loss: 2.079397, Acc: 12.5248
Test_: Total Testing Acc: 12.5000
Time taken = 80.8231 s

Train: Epoch:1, Batch:19, Loss:2.079542, Acc:9.3750
Train_: Avg Training Loss: 2.075433, Avg Training Acc: 15.069901
Val_: Total Validation Loss: 2.079680, Acc: 12.1404
Test_: Total Testing Acc: 12.4628
Time taken = 76.1498 s

Train: Epoch:2, Batch:19, Loss:2.073436, Acc:17.1875
Train_: Avg Training Loss: 2.072039, Avg Training Acc: 16.652961
Val_: Total Validation Loss: 2.079877, Acc: 12.3760
Test_: Total Testing Acc: 12.4504
Time taken = 67.6931 s

Train: Epoch:3, Batch:19, Loss:2.068140, Acc:12.5000
Train_: Avg Training Loss: 2.069022, Avg Training Acc: 17.228618
Val_: Total Validation Loss: 2.079839, Acc: 12.5992
Test_: Total Testing Acc: 12.5620
Time taken = 67.4743 s

Train: Epoch:4, Batch:19, Loss:2.069503, Acc:16.4062
Train_: Avg Training Loss: 2.064966,

Time taken = 71.8524 s

Train: Epoch:36, Batch:19, Loss:1.556694, Acc:73.4375
Train_: Avg Training Loss: 1.533084, Avg Training Acc: 76.624178
Val_: Total Validation Loss: 2.106267, Acc: 13.5169
Test_: Total Testing Acc: 13.1944
Time taken = 80.0434 s

Train: Epoch:37, Batch:19, Loss:1.543400, Acc:75.0000
Train_: Avg Training Loss: 1.526384, Avg Training Acc: 77.138158
Val_: Total Validation Loss: 2.106728, Acc: 13.3309
Test_: Total Testing Acc: 13.2316
Time taken = 71.3393 s

Train: Epoch:38, Batch:19, Loss:1.541722, Acc:75.7812
Train_: Avg Training Loss: 1.522013, Avg Training Acc: 77.487664
Val_: Total Validation Loss: 2.108264, Acc: 13.2192
Test_: Total Testing Acc: 13.4301
Time taken = 69.0451 s

Train: Epoch:39, Batch:19, Loss:1.536913, Acc:76.5625
Train_: Avg Training Loss: 1.517619, Avg Training Acc: 77.611020
Val_: Total Validation Loss: 2.107460, Acc: 13.4053
Test_: Total Testing Acc: 13.2937
Time taken = 68.4543 s

Train: Epoch:40, Batch:19, Loss:1.529518, Acc:76.5625
Train_

Test_: Total Testing Acc: 13.3433
Time taken = 79.5730 s

Train: Epoch:72, Batch:19, Loss:1.368286, Acc:91.4062
Train_: Avg Training Loss: 1.367025, Avg Training Acc: 91.673520
Val_: Total Validation Loss: 2.113451, Acc: 13.1820
Test_: Total Testing Acc: 13.4921
Time taken = 68.2577 s

Train: Epoch:73, Batch:19, Loss:1.370385, Acc:92.1875
Train_: Avg Training Loss: 1.367865, Avg Training Acc: 91.694079
Val_: Total Validation Loss: 2.113555, Acc: 12.9092
Test_: Total Testing Acc: 13.4425
Time taken = 68.4919 s

Train: Epoch:74, Batch:19, Loss:1.359950, Acc:92.1875
Train_: Avg Training Loss: 1.365706, Avg Training Acc: 91.735197
Val_: Total Validation Loss: 2.115178, Acc: 12.8100
Test_: Total Testing Acc: 13.7153
Time taken = 66.0902 s

Train: Epoch:75, Batch:19, Loss:1.364491, Acc:92.1875
Train_: Avg Training Loss: 1.364802, Avg Training Acc: 91.817434
Val_: Total Validation Loss: 2.114413, Acc: 13.2440
Test_: Total Testing Acc: 13.4797
Time taken = 67.2633 s

Train: Epoch:76, Batch:19,

In [ ]:
# Re-running for 40% of the data
SAVE_FILE = "last_take1_cnn_mlp_40DAT_" + time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime()) + "_" + str(args.perc_train)

for epoch in range(0, args.epochs):
    t0 = time.time()
    train(epoch, SAVE_FILE)
    avg_loss, avg_acc = validate(epoch, SAVE_FILE)
    test_acc = test(epoch, SAVE_FILE)
    model.save_model(args.save, epoch, avg_acc, avg_loss)
    print("Time taken = {:.4f} s\n".format(time.time() - t0))

Train: Epoch:0, Batch:19, Loss:2.083294, Acc:7.8125
Train: Epoch:0, Batch:39, Loss:2.083657, Acc:9.3750
Train: Epoch:0, Batch:59, Loss:2.078551, Acc:14.0625
Train_: Avg Training Loss: 2.079822, Avg Training Acc: 12.312500
Val_: Total Validation Loss: 2.079904, Acc: 12.3512
Test_: Total Testing Acc: 12.2396
Time taken = 69.3532 s

Train: Epoch:1, Batch:19, Loss:2.077670, Acc:17.1875
Train: Epoch:1, Batch:39, Loss:2.075178, Acc:17.9688
Train: Epoch:1, Batch:59, Loss:2.079865, Acc:16.4062
Train_: Avg Training Loss: 2.076895, Avg Training Acc: 15.250000
Val_: Total Validation Loss: 2.079776, Acc: 12.7356
Test_: Total Testing Acc: 12.6240
Time taken = 69.3063 s

Train: Epoch:2, Batch:19, Loss:2.072519, Acc:19.5312
Train: Epoch:2, Batch:39, Loss:2.075179, Acc:20.3125
Train: Epoch:2, Batch:59, Loss:2.071651, Acc:17.9688
Train_: Avg Training Loss: 2.073834, Avg Training Acc: 16.322917
Val_: Total Validation Loss: 2.079613, Acc: 12.8472
Test_: Total Testing Acc: 12.7232
Time taken = 70.8590 s



Train_: Avg Training Loss: 1.682341, Avg Training Acc: 63.333333
Val_: Total Validation Loss: 2.081356, Acc: 15.8978
Test_: Total Testing Acc: 16.5675
Time taken = 69.4363 s

Train: Epoch:25, Batch:19, Loss:1.640720, Acc:70.3125
Train: Epoch:25, Batch:39, Loss:1.633225, Acc:69.5312
Train: Epoch:25, Batch:59, Loss:1.614920, Acc:69.5312
Train_: Avg Training Loss: 1.669306, Avg Training Acc: 64.385417
Val_: Total Validation Loss: 2.083063, Acc: 15.9474
Test_: Total Testing Acc: 15.9846
Time taken = 68.0271 s

Train: Epoch:26, Batch:19, Loss:1.640423, Acc:67.1875
Train: Epoch:26, Batch:39, Loss:1.618396, Acc:70.3125
Train: Epoch:26, Batch:59, Loss:1.623930, Acc:70.3125
Train_: Avg Training Loss: 1.656338, Avg Training Acc: 65.791667
Val_: Total Validation Loss: 2.083491, Acc: 16.0962
Test_: Total Testing Acc: 15.7366
Time taken = 68.4210 s

Train: Epoch:27, Batch:19, Loss:1.632423, Acc:71.8750
Train: Epoch:27, Batch:39, Loss:1.603314, Acc:70.3125
Train: Epoch:27, Batch:59, Loss:1.599696, A

Test_: Total Testing Acc: 15.9970
Time taken = 71.0778 s

Train: Epoch:49, Batch:19, Loss:1.493375, Acc:78.9062
Train: Epoch:49, Batch:39, Loss:1.473728, Acc:81.2500
Train: Epoch:49, Batch:59, Loss:1.467386, Acc:82.0312
Train_: Avg Training Loss: 1.500397, Avg Training Acc: 79.093750
Val_: Total Validation Loss: 2.092769, Acc: 16.2326
Test_: Total Testing Acc: 15.8482
Time taken = 72.4861 s

Train: Epoch:50, Batch:19, Loss:1.499824, Acc:78.9062
Train: Epoch:50, Batch:39, Loss:1.474673, Acc:82.0312
Train: Epoch:50, Batch:59, Loss:1.476684, Acc:82.8125
Train_: Avg Training Loss: 1.497446, Avg Training Acc: 79.177083
Val_: Total Validation Loss: 2.093042, Acc: 16.2822
Test_: Total Testing Acc: 15.7738
Time taken = 72.5813 s

Train: Epoch:51, Batch:19, Loss:1.503402, Acc:79.6875
Train: Epoch:51, Batch:39, Loss:1.457277, Acc:83.5938
Train: Epoch:51, Batch:59, Loss:1.479233, Acc:82.0312
Train_: Avg Training Loss: 1.495803, Avg Training Acc: 79.364583
Val_: Total Validation Loss: 2.093987, Ac

Train: Epoch:73, Batch:39, Loss:1.433278, Acc:85.1562
Train: Epoch:73, Batch:59, Loss:1.429739, Acc:85.1562
Train_: Avg Training Loss: 1.456057, Avg Training Acc: 82.322917
Val_: Total Validation Loss: 2.098660, Acc: 15.7118
Test_: Total Testing Acc: 15.1910
Time taken = 71.4187 s

Train: Epoch:74, Batch:19, Loss:1.467954, Acc:80.4688
Train: Epoch:74, Batch:39, Loss:1.444043, Acc:82.8125
Train: Epoch:74, Batch:59, Loss:1.434920, Acc:85.1562
Train_: Avg Training Loss: 1.455150, Avg Training Acc: 82.291667
Val_: Total Validation Loss: 2.097907, Acc: 15.7986
Test_: Total Testing Acc: 15.5878
Time taken = 72.7851 s

Train: Epoch:75, Batch:19, Loss:1.472006, Acc:80.4688
Train: Epoch:75, Batch:39, Loss:1.425968, Acc:85.1562
Train: Epoch:75, Batch:59, Loss:1.428365, Acc:85.1562
Train_: Avg Training Loss: 1.452988, Avg Training Acc: 82.468750
Val_: Total Validation Loss: 2.098520, Acc: 15.9102
Test_: Total Testing Acc: 15.8730
Time taken = 71.7532 s

Train: Epoch:76, Batch:19, Loss:1.475458, A

Train_: Avg Training Loss: 1.438305, Avg Training Acc: 83.645833
Val_: Total Validation Loss: 2.101453, Acc: 15.6746
Test_: Total Testing Acc: 15.8606
Time taken = 68.7100 s

Train: Epoch:98, Batch:19, Loss:1.453134, Acc:82.0312
Train: Epoch:98, Batch:39, Loss:1.392809, Acc:88.2812
Train: Epoch:98, Batch:59, Loss:1.416269, Acc:85.9375
Train_: Avg Training Loss: 1.437857, Avg Training Acc: 83.687500
Val_: Total Validation Loss: 2.099680, Acc: 15.6994
Test_: Total Testing Acc: 16.0714
Time taken = 69.9789 s

Train: Epoch:99, Batch:19, Loss:1.456259, Acc:82.0312
Train: Epoch:99, Batch:39, Loss:1.399724, Acc:87.5000
Train: Epoch:99, Batch:59, Loss:1.414619, Acc:85.9375
Train_: Avg Training Loss: 1.437989, Avg Training Acc: 83.677083
Val_: Total Validation Loss: 2.101172, Acc: 15.6126
Test_: Total Testing Acc: 16.0838
Time taken = 68.9007 s



In [ ]:
# Re-running for 60% of the data
SAVE_FILE = "last_take1_cnn_mlp_60DAT_" + time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime()) + "_" + str(args.perc_train)

for epoch in range(0, args.epochs):
    t0 = time.time()
    train(epoch, SAVE_FILE)
    avg_loss, avg_acc = validate(epoch, SAVE_FILE)
    test_acc = test(epoch, SAVE_FILE)
    model.save_model(args.save, epoch, avg_acc, avg_loss)
    print("Time taken = {:.4f} s\n".format(time.time() - t0))

Train: Epoch:0, Batch:19, Loss:2.081889, Acc:10.1562
Train: Epoch:0, Batch:39, Loss:2.079479, Acc:8.5938
Train: Epoch:0, Batch:59, Loss:2.081330, Acc:11.7188
Train: Epoch:0, Batch:79, Loss:2.073344, Acc:13.2812
Train: Epoch:0, Batch:99, Loss:2.080355, Acc:10.9375
Train_: Avg Training Loss: 2.079745, Avg Training Acc: 12.506914
Val_: Total Validation Loss: 2.079279, Acc: 12.3884
Test_: Total Testing Acc: 12.5992
Time taken = 73.3287 s

Train: Epoch:1, Batch:19, Loss:2.078767, Acc:14.0625
Train: Epoch:1, Batch:39, Loss:2.081256, Acc:11.7188
Train: Epoch:1, Batch:59, Loss:2.075505, Acc:15.6250
Train: Epoch:1, Batch:79, Loss:2.073528, Acc:20.3125
Train: Epoch:1, Batch:99, Loss:2.071070, Acc:21.0938
Train_: Avg Training Loss: 2.076532, Avg Training Acc: 14.823009
Val_: Total Validation Loss: 2.078953, Acc: 12.3016
Test_: Total Testing Acc: 12.6240
Time taken = 75.9900 s

Train: Epoch:2, Batch:19, Loss:2.076903, Acc:9.3750
Train: Epoch:2, Batch:39, Loss:2.077100, Acc:14.8438
Train: Epoch:2, 

Train_: Avg Training Loss: 1.798874, Avg Training Acc: 50.400996
Val_: Total Validation Loss: 2.073435, Acc: 16.6543
Test_: Total Testing Acc: 17.0883
Time taken = 74.6792 s

Train: Epoch:19, Batch:19, Loss:1.830844, Acc:45.3125
Train: Epoch:19, Batch:39, Loss:1.819584, Acc:46.0938
Train: Epoch:19, Batch:59, Loss:1.774784, Acc:51.5625
Train: Epoch:19, Batch:79, Loss:1.765310, Acc:54.6875
Train: Epoch:19, Batch:99, Loss:1.721901, Acc:57.8125
Train_: Avg Training Loss: 1.783632, Avg Training Acc: 52.018805
Val_: Total Validation Loss: 2.073096, Acc: 16.8775
Test_: Total Testing Acc: 17.3115
Time taken = 77.1756 s

Train: Epoch:20, Batch:19, Loss:1.811089, Acc:47.6562
Train: Epoch:20, Batch:39, Loss:1.799762, Acc:47.6562
Train: Epoch:20, Batch:59, Loss:1.763621, Acc:53.1250
Train: Epoch:20, Batch:79, Loss:1.763075, Acc:56.2500
Train: Epoch:20, Batch:99, Loss:1.742324, Acc:56.2500
Train_: Avg Training Loss: 1.771155, Avg Training Acc: 53.228706
Val_: Total Validation Loss: 2.072339, Acc: 1

Train: Epoch:37, Batch:39, Loss:1.611693, Acc:69.5312
Train: Epoch:37, Batch:59, Loss:1.613613, Acc:67.1875
Train: Epoch:37, Batch:79, Loss:1.630846, Acc:67.1875
Train: Epoch:37, Batch:99, Loss:1.616549, Acc:66.4062
Train_: Avg Training Loss: 1.617647, Avg Training Acc: 68.010232
Val_: Total Validation Loss: 2.094665, Acc: 15.9598
Test_: Total Testing Acc: 16.5179
Time taken = 75.2081 s

Train: Epoch:38, Batch:19, Loss:1.677093, Acc:60.9375
Train: Epoch:38, Batch:39, Loss:1.593133, Acc:71.0938
Train: Epoch:38, Batch:59, Loss:1.611124, Acc:68.7500
Train: Epoch:38, Batch:79, Loss:1.619286, Acc:67.1875
Train: Epoch:38, Batch:99, Loss:1.586443, Acc:71.8750
Train_: Avg Training Loss: 1.611431, Avg Training Acc: 68.549502
Val_: Total Validation Loss: 2.090959, Acc: 16.2574
Test_: Total Testing Acc: 16.2574
Time taken = 73.3987 s

Train: Epoch:39, Batch:19, Loss:1.671201, Acc:62.5000
Train: Epoch:39, Batch:39, Loss:1.586244, Acc:71.8750
Train: Epoch:39, Batch:59, Loss:1.597252, Acc:69.5312
Tr

Train_: Avg Training Loss: 1.544599, Avg Training Acc: 74.357024
Val_: Total Validation Loss: 2.093860, Acc: 16.0218
Test_: Total Testing Acc: 16.1706
Time taken = 74.5229 s

Train: Epoch:56, Batch:19, Loss:1.625704, Acc:67.1875
Train: Epoch:56, Batch:39, Loss:1.532774, Acc:75.7812
Train: Epoch:56, Batch:59, Loss:1.556927, Acc:74.2188
Train: Epoch:56, Batch:79, Loss:1.567742, Acc:71.8750
Train: Epoch:56, Batch:99, Loss:1.512330, Acc:77.3438
Train_: Avg Training Loss: 1.541782, Avg Training Acc: 74.612832
Val_: Total Validation Loss: 2.094929, Acc: 16.2078
Test_: Total Testing Acc: 16.2946
Time taken = 71.3925 s

Train: Epoch:57, Batch:19, Loss:1.609771, Acc:68.7500
Train: Epoch:57, Batch:39, Loss:1.524893, Acc:76.5625
Train: Epoch:57, Batch:59, Loss:1.544289, Acc:72.6562
Train: Epoch:57, Batch:79, Loss:1.562912, Acc:71.8750
Train: Epoch:57, Batch:99, Loss:1.503637, Acc:78.1250
Train_: Avg Training Loss: 1.538022, Avg Training Acc: 74.930863
Val_: Total Validation Loss: 2.093908, Acc: 1

Train: Epoch:74, Batch:39, Loss:1.485259, Acc:79.6875
Train: Epoch:74, Batch:59, Loss:1.496121, Acc:78.9062
Train: Epoch:74, Batch:79, Loss:1.528292, Acc:75.0000
Train: Epoch:74, Batch:99, Loss:1.470365, Acc:82.0312
Train_: Avg Training Loss: 1.505945, Avg Training Acc: 77.606471
Val_: Total Validation Loss: 2.089780, Acc: 16.8403
Test_: Total Testing Acc: 16.3442
Time taken = 84.5638 s

Train: Epoch:75, Batch:19, Loss:1.550885, Acc:72.6562
Train: Epoch:75, Batch:39, Loss:1.477908, Acc:80.4688
Train: Epoch:75, Batch:59, Loss:1.509460, Acc:77.3438
Train: Epoch:75, Batch:79, Loss:1.531809, Acc:75.0000
Train: Epoch:75, Batch:99, Loss:1.461124, Acc:82.0312
Train_: Avg Training Loss: 1.504749, Avg Training Acc: 77.627212
Val_: Total Validation Loss: 2.094815, Acc: 16.2574
Test_: Total Testing Acc: 16.2822
Time taken = 78.6190 s

Train: Epoch:76, Batch:19, Loss:1.543059, Acc:74.2188
Train: Epoch:76, Batch:39, Loss:1.489546, Acc:79.6875
Train: Epoch:76, Batch:59, Loss:1.496731, Acc:78.1250
Tr

Train_: Avg Training Loss: 1.491104, Avg Training Acc: 78.726493
Val_: Total Validation Loss: 2.094412, Acc: 16.5551
Test_: Total Testing Acc: 16.1706
Time taken = 74.8922 s

Train: Epoch:93, Batch:19, Loss:1.517095, Acc:76.5625
Train: Epoch:93, Batch:39, Loss:1.463328, Acc:81.2500
Train: Epoch:93, Batch:59, Loss:1.482734, Acc:79.6875
Train: Epoch:93, Batch:79, Loss:1.518084, Acc:75.7812
Train: Epoch:93, Batch:99, Loss:1.461020, Acc:82.0312
Train_: Avg Training Loss: 1.487833, Avg Training Acc: 78.954646
Val_: Total Validation Loss: 2.091462, Acc: 16.6915
Test_: Total Testing Acc: 16.3566
Time taken = 73.1848 s

Train: Epoch:94, Batch:19, Loss:1.517809, Acc:76.5625
Train: Epoch:94, Batch:39, Loss:1.475983, Acc:80.4688
Train: Epoch:94, Batch:59, Loss:1.484905, Acc:78.9062
Train: Epoch:94, Batch:79, Loss:1.518040, Acc:75.7812
Train: Epoch:94, Batch:99, Loss:1.458972, Acc:82.0312
Train_: Avg Training Loss: 1.486749, Avg Training Acc: 79.016869
Val_: Total Validation Loss: 2.094924, Acc: 1

In [ ]:
# Re-running for 80% of the data
SAVE_FILE = "last_take1_cnn_mlp_80DAT_" + time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime()) + "_" + str(args.perc_train)

for epoch in range(0, args.epochs):
    t0 = time.time()
    train(epoch, SAVE_FILE)
    avg_loss, avg_acc = validate(epoch, SAVE_FILE)
    test_acc = test(epoch, SAVE_FILE)
    model.save_model(args.save, epoch, avg_acc, avg_loss)
    print("Time taken = {:.4f} s\n".format(time.time() - t0))

Train: Epoch:0, Batch:19, Loss:2.075218, Acc:17.1875
Train: Epoch:0, Batch:39, Loss:2.085316, Acc:10.9375
Train: Epoch:0, Batch:59, Loss:2.081535, Acc:12.5000
Train: Epoch:0, Batch:79, Loss:2.074245, Acc:17.9688
Train: Epoch:0, Batch:99, Loss:2.081696, Acc:12.5000
Train: Epoch:0, Batch:119, Loss:2.082397, Acc:9.3750
Train: Epoch:0, Batch:139, Loss:2.075655, Acc:14.0625
Train_: Avg Training Loss: 2.079659, Avg Training Acc: 12.593750
Val_: Total Validation Loss: 2.079115, Acc: 13.0332
Test_: Total Testing Acc: 12.1032
Time taken = 79.3132 s

Train: Epoch:1, Batch:19, Loss:2.077569, Acc:11.7188
Train: Epoch:1, Batch:39, Loss:2.077389, Acc:14.8438
Train: Epoch:1, Batch:59, Loss:2.076812, Acc:13.2812
Train: Epoch:1, Batch:79, Loss:2.074974, Acc:15.6250
Train: Epoch:1, Batch:99, Loss:2.080638, Acc:15.6250
Train: Epoch:1, Batch:119, Loss:2.076798, Acc:13.2812
Train: Epoch:1, Batch:139, Loss:2.073241, Acc:13.2812
Train_: Avg Training Loss: 2.076934, Avg Training Acc: 14.484375
Val_: Total Val

Test_: Total Testing Acc: 17.8943
Time taken = 77.1814 s

Train: Epoch:15, Batch:19, Loss:1.894094, Acc:42.1875
Train: Epoch:15, Batch:39, Loss:1.867389, Acc:39.8438
Train: Epoch:15, Batch:59, Loss:1.844158, Acc:44.5312
Train: Epoch:15, Batch:79, Loss:1.835647, Acc:47.6562
Train: Epoch:15, Batch:99, Loss:1.810809, Acc:52.3438
Train: Epoch:15, Batch:119, Loss:1.795103, Acc:53.1250
Train: Epoch:15, Batch:139, Loss:1.787336, Acc:53.9062
Train_: Avg Training Loss: 1.842608, Avg Training Acc: 45.718750
Val_: Total Validation Loss: 2.062038, Acc: 18.0060
Test_: Total Testing Acc: 18.0556
Time taken = 77.4878 s

Train: Epoch:16, Batch:19, Loss:1.870307, Acc:42.9688
Train: Epoch:16, Batch:39, Loss:1.858623, Acc:42.1875
Train: Epoch:16, Batch:59, Loss:1.843104, Acc:43.7500
Train: Epoch:16, Batch:79, Loss:1.811533, Acc:46.0938
Train: Epoch:16, Batch:99, Loss:1.776788, Acc:55.4688
Train: Epoch:16, Batch:119, Loss:1.788991, Acc:50.7812
Train: Epoch:16, Batch:139, Loss:1.776856, Acc:52.3438
Train_:

Train_: Avg Training Loss: 1.686048, Avg Training Acc: 61.166667
Val_: Total Validation Loss: 2.076898, Acc: 17.5471
Test_: Total Testing Acc: 18.2292
Time taken = 77.3047 s

Train: Epoch:30, Batch:19, Loss:1.697891, Acc:59.3750
Train: Epoch:30, Batch:39, Loss:1.681077, Acc:62.5000
Train: Epoch:30, Batch:59, Loss:1.653708, Acc:63.2812
Train: Epoch:30, Batch:79, Loss:1.653798, Acc:67.1875
Train: Epoch:30, Batch:99, Loss:1.649511, Acc:65.6250
Train: Epoch:30, Batch:119, Loss:1.634832, Acc:66.4062
Train: Epoch:30, Batch:139, Loss:1.629731, Acc:67.9688
Train_: Avg Training Loss: 1.677884, Avg Training Acc: 62.177083
Val_: Total Validation Loss: 2.076439, Acc: 17.7455
Test_: Total Testing Acc: 17.7579
Time taken = 82.4469 s

Train: Epoch:31, Batch:19, Loss:1.671432, Acc:61.7188
Train: Epoch:31, Batch:39, Loss:1.661183, Acc:65.6250
Train: Epoch:31, Batch:59, Loss:1.651843, Acc:64.8438
Train: Epoch:31, Batch:79, Loss:1.614687, Acc:67.9688
Train: Epoch:31, Batch:99, Loss:1.659020, Acc:64.0625


Train: Epoch:44, Batch:119, Loss:1.557557, Acc:71.8750
Train: Epoch:44, Batch:139, Loss:1.554068, Acc:74.2188
Train_: Avg Training Loss: 1.607454, Avg Training Acc: 68.302083
Val_: Total Validation Loss: 2.092326, Acc: 16.6295
Test_: Total Testing Acc: 17.5223
Time taken = 78.1683 s

Train: Epoch:45, Batch:19, Loss:1.608628, Acc:68.7500
Train: Epoch:45, Batch:39, Loss:1.580780, Acc:70.3125
Train: Epoch:45, Batch:59, Loss:1.623269, Acc:67.1875
Train: Epoch:45, Batch:79, Loss:1.556501, Acc:72.6562
Train: Epoch:45, Batch:99, Loss:1.595776, Acc:69.5312
Train: Epoch:45, Batch:119, Loss:1.553929, Acc:73.4375
Train: Epoch:45, Batch:139, Loss:1.560944, Acc:71.8750
Train_: Avg Training Loss: 1.604771, Avg Training Acc: 68.468750
Val_: Total Validation Loss: 2.082834, Acc: 17.4975
Test_: Total Testing Acc: 17.9563
Time taken = 91.3914 s

Train: Epoch:46, Batch:19, Loss:1.590670, Acc:69.5312
Train: Epoch:46, Batch:39, Loss:1.596688, Acc:70.3125
Train: Epoch:46, Batch:59, Loss:1.604585, Acc:68.750

Train: Epoch:59, Batch:79, Loss:1.511395, Acc:78.1250
Train: Epoch:59, Batch:99, Loss:1.558136, Acc:71.8750
Train: Epoch:59, Batch:119, Loss:1.527412, Acc:75.7812
Train: Epoch:59, Batch:139, Loss:1.530534, Acc:73.4375
Train_: Avg Training Loss: 1.568878, Avg Training Acc: 71.526042
Val_: Total Validation Loss: 2.095788, Acc: 16.4931
Test_: Total Testing Acc: 16.5055
Time taken = 86.6005 s

Train: Epoch:60, Batch:19, Loss:1.575207, Acc:70.3125
Train: Epoch:60, Batch:39, Loss:1.542405, Acc:75.7812
Train: Epoch:60, Batch:59, Loss:1.579316, Acc:71.0938
Train: Epoch:60, Batch:79, Loss:1.498640, Acc:78.9062
Train: Epoch:60, Batch:99, Loss:1.569853, Acc:71.8750
Train: Epoch:60, Batch:119, Loss:1.539916, Acc:74.2188
Train: Epoch:60, Batch:139, Loss:1.523422, Acc:76.5625
Train_: Avg Training Loss: 1.566796, Avg Training Acc: 71.708333
Val_: Total Validation Loss: 2.093507, Acc: 16.7659
Test_: Total Testing Acc: 17.0387
Time taken = 91.3385 s

Train: Epoch:61, Batch:19, Loss:1.566861, Acc:71.875

Train: Epoch:74, Batch:39, Loss:1.532229, Acc:75.0000
Train: Epoch:74, Batch:59, Loss:1.550270, Acc:73.4375
Train: Epoch:74, Batch:79, Loss:1.478812, Acc:80.4688
Train: Epoch:74, Batch:99, Loss:1.543071, Acc:74.2188
Train: Epoch:74, Batch:119, Loss:1.524355, Acc:75.0000
Train: Epoch:74, Batch:139, Loss:1.536190, Acc:74.2188
Train_: Avg Training Loss: 1.543698, Avg Training Acc: 73.682292
Val_: Total Validation Loss: 2.102671, Acc: 15.7614
Test_: Total Testing Acc: 17.0759
Time taken = 88.0570 s

Train: Epoch:75, Batch:19, Loss:1.540050, Acc:74.2188
Train: Epoch:75, Batch:39, Loss:1.510715, Acc:77.3438
Train: Epoch:75, Batch:59, Loss:1.546932, Acc:74.2188
Train: Epoch:75, Batch:79, Loss:1.485722, Acc:79.6875
Train: Epoch:75, Batch:99, Loss:1.535172, Acc:74.2188
Train: Epoch:75, Batch:119, Loss:1.518724, Acc:76.5625
Train: Epoch:75, Batch:139, Loss:1.512524, Acc:75.0000
Train_: Avg Training Loss: 1.543601, Avg Training Acc: 73.666667
Val_: Total Validation Loss: 2.102155, Acc: 15.7490
Te

Test_: Total Testing Acc: 17.9812
Time taken = 83.1828 s

Train: Epoch:89, Batch:19, Loss:1.532712, Acc:74.2188
Train: Epoch:89, Batch:39, Loss:1.489173, Acc:78.9062
Train: Epoch:89, Batch:59, Loss:1.535770, Acc:74.2188
Train: Epoch:89, Batch:79, Loss:1.462922, Acc:81.2500
Train: Epoch:89, Batch:99, Loss:1.525390, Acc:75.0000
Train: Epoch:89, Batch:119, Loss:1.512441, Acc:76.5625
Train: Epoch:89, Batch:139, Loss:1.459853, Acc:81.2500
Train_: Avg Training Loss: 1.529428, Avg Training Acc: 74.843750
Val_: Total Validation Loss: 2.097228, Acc: 16.3442
Test_: Total Testing Acc: 18.1052
Time taken = 88.4839 s

Train: Epoch:90, Batch:19, Loss:1.519776, Acc:75.7812
Train: Epoch:90, Batch:39, Loss:1.503814, Acc:78.1250
Train: Epoch:90, Batch:59, Loss:1.527809, Acc:75.0000
Train: Epoch:90, Batch:79, Loss:1.453531, Acc:82.0312
Train: Epoch:90, Batch:99, Loss:1.522279, Acc:75.7812
Train: Epoch:90, Batch:119, Loss:1.512341, Acc:76.5625
Train: Epoch:90, Batch:139, Loss:1.474794, Acc:79.6875
Train_:

#### Test on Center Single, 2x2 Grid, Left-Right, Out-In

In [ ]:
testset = dataset(args.path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))

In [ ]:
center_single_fnames = [x for x in testset.file_names if '/center_single/' in x]
distribute_four_fnames = [x for x in testset.file_names if '/distribute_four/' in x]
in_distribute_four_out_center_single_fnames = \
    [x for x in testset.file_names if '/in_distribute_four_out_center_single/' in x]
left_center_single_right_center_single_fnames = \
    [x for x in testset.file_names if '/left_center_single_right_center_single/' in x]

In [ ]:
print(len(center_single_fnames), \
len(distribute_four_fnames), \
len(in_distribute_four_out_center_single_fnames), \
len(left_center_single_right_center_single_fnames))

2000 2000 2000 2000


In [ ]:
cs_set = dataset(args.path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))
cs_set.file_names = center_single_fnames
cs_testloader = DataLoader(cs_set, batch_size=args.batch_size, shuffle=False, num_workers=80)

df_set = dataset(args.path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))
df_set.file_names = distribute_four_fnames
df_testloader = DataLoader(df_set, batch_size=args.batch_size, shuffle=False, num_workers=80)

idfo_set = dataset(args.path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))
idfo_set.file_names = in_distribute_four_out_center_single_fnames
idfo_testloader = DataLoader(idfo_set, batch_size=args.batch_size, shuffle=False, num_workers=80)

lcsr_set = dataset(args.path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))
lcsr_set.file_names = left_center_single_right_center_single_fnames
lcsr_testloader = DataLoader(lcsr_set, batch_size=args.batch_size, shuffle=False, num_workers=80)

In [ ]:
def test_sep(epoch, tl, save_file):
    model.eval()
    accuracy = 0

    acc_all = 0.0
    counter = 0
    for batch_idx, (image, target, meta_target, meta_structure, embedding, indicator) in enumerate(tl):
        counter += 1
        if args.cuda:
            image = image.cuda()
            target = target.cuda()
            meta_target = meta_target.cuda()
            meta_structure = meta_structure.cuda()
            embedding = embedding.cuda()
            indicator = indicator.cuda()
        acc = model.test_(image, target, meta_target, meta_structure, embedding, indicator)
        # print('Test: Epoch:{}, Batch:{}, Acc:{:.4f}.'.format(epoch, batch_idx, acc))  
        acc_all += acc
    if counter > 0:
        save_str = "Test_: Total Testing Acc: {:.4f}".format(acc_all / float(counter))
        print(save_str)
        with open(save_file, 'a') as f:
            f.write(save_str + "\n")
    return acc_all/float(counter)

In [ ]:
#Trained on 4 Configurations
SAVE_FILE = "cnn_mlp_" + time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime()) + "_" + str(args.perc_train)
SAVE_FILE_SEP = SAVE_FILE + "_sep"

test_acc_cs = test_sep(epoch, cs_testloader, SAVE_FILE_SEP)
test_acc_df = test_sep(epoch, df_testloader, SAVE_FILE_SEP)
test_acc_idfo = test_sep(epoch, idfo_testloader, SAVE_FILE_SEP)
test_acc_lcsr = test_sep(epoch, lcsr_testloader, SAVE_FILE_SEP)

Test_: Total Testing Acc: 20.5371
Test_: Total Testing Acc: 20.1758
Test_: Total Testing Acc: 22.7344
Test_: Total Testing Acc: 22.5586
